# Sentiment Analysis: Donald Trump and Joe Biden Reelection Run in 2024

## Description

This part of the project will be the develop of the code to gather the real time data from the Twitter API. The data will be used to perform a sentiment analysis of the tweets about Donald Trump and Joe Biden reelection run in 2024.



## Gathering Data

In [1]:
import json
with open("C:/Users/diazi/.secret/twitter_api.json") as f:
    keys = json.load(f)

keys.keys()

dict_keys(['consumer-key', 'consumer-key-secret', 'access-token', 'access-token-secret', 'bearer-token'])

In [2]:
## tokens
bearer_token = keys['bearer-token']
consumer_key = keys['consumer-key']
consumer_secret = keys['consumer-key-secret']
access_key = keys['access-token']
access_secret = keys['access-token-secret']

**Setting up the connection to the Twitter API**

In [5]:
import tweepy
import csv
import json
import sys
import datetime

In [7]:
file_name = 'elections2024.csv'

In [8]:
# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open(file_name, "a", encoding='utf-8') as f:
            #para csv
            writer = csv.writer(f)
            writer.writerow([status.created_at,status.id, status.user.screen_name,is_retweet,is_quote,text,quoted_text, status.entities, status.geo,
                             status.coordinates, status.place, status.retweet_count, status.favorite_count, status.lang
                             ])
            #para txt
            #f.write("%s,%s,%s,%s,%s,%s\n" % (status.created_at,status.user.screen_name,is_retweet,is_quote,text,quoted_text))

    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        pass

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')

    #with open("data/first100days.txt", "w", encoding='utf-8') as f:
    #    f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    with open(file_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['date', 'id','screen_name', 'is_retweet','is_quote', 'text', 'quoted_text', 'entities', 'geo',
                         'coordinates', 'place', 'retweet_count', 'favorite_count', 'lang'
                         ])

    tags = ['#elections2024', '#trump2024', '#biden2024', '#donaldtrump2024', '#joebiden2024']
    stream.filter(track=tags)

Encountered streaming error ( 404 )
Encountered streaming error ( 404 )
Encountered streaming error ( 404 )
Encountered streaming error ( 404 )
Encountered streaming error ( 404 )


KeyboardInterrupt: 

In [10]:
import csv
import requests
from datetime import datetime, timedelta
import json

class TwitterData:
    def __init__(self, bearer_token):
        self.bearer_token = bearer_token
        self.base_url = "https://api.twitter.com/2/tweets/sample/stream"
        self.headers = {"Authorization": f"Bearer {self.bearer_token}"}
        self.filename = f"twitter_data_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
        self.fieldnames = ['id', 'text', 'attachments', 'author_id', 'context_annotations',
                           'conversation_id', 'created_at', 'entities', 'geo', 'in_reply_to_user_id',
                           'lang', 'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
                           'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
                           'source', 'withheld']
        self.start_time = datetime.now()

    def get_data(self):
        with open(self.filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=self.fieldnames)
            writer.writeheader()
            response = requests.get(self.base_url, headers=self.headers, stream=True)
            for response_line in response.iter_lines():
                if datetime.now() - self.start_time >= timedelta(hours=1):
                    self.filename = f"twitter_data_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
                    file.close()
                    file = open(self.filename, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=self.fieldnames)
                    writer.writeheader()
                    self.start_time = datetime.now()
                if response_line:
                    tweet = response_line.json()
                    writer.writerow({field: tweet.get(field) for field in self.fieldnames})

In [11]:
if __name__ == "__main__":
    twitter_data = TwitterData(bearer_token)
    twitter_data.get_data()

AttributeError: 'bytes' object has no attribute 'json'